# Classes and Objects
- **Beyond Built-ins:** Python lets you define your own data types using `class`.
- **Class:** A blueprint or template for creating objects. Defines attributes (data) and methods (behavior). Convention: `PascalCase` names (`MyClass`).
- **Object (Instance):** A specific item created from a class blueprint. Each object has its own set of attribute values but shares the methods defined by the class. `obj1 = MyClass()`, `obj2 = MyClass()`. `obj1` and `obj2` are distinct objects.

### Keywords
- **Instantiate:** Create an instance of a class.
- **Attribute:** A variable bound to an instance of a class.
- **Method:** A function in a class.
- **self:** Reference to the instance of a class

## Defining a Class & `__init__` (The Constructor)
- **`__init__(self, ...)`:** Special method for initialization. `self` is always the first parameter and represents the instance itself. Other parameters receive arguments passed during object creation.
- **Instance Attributes (`self.x = ...`):** Data attached to *this specific object*. Created inside methods (usually `__init__`) using `self.attribute_name = value`.

In [3]:
class ServiceMonitor:
    """Provides service checks for a single service"""
    def __init__(self, service_name, port):
        """Initializes the monitor for a specific service.

        Args:
            service_name (str): the name of the service.
            port (int): the port to use for checks.
        """
        print(f"Initializing monitor for service {service_name} on port {port}.")
        self.service = service_name
        self.port = port
        self.is_alive = False

## Creating Instances (Objects)
- **Mechanism:** Call the class name as if it were a function, passing any arguments required by `__init__` (after `self`).
- Python automatically creates the object and passes it as `self` to `__init__`.

In [4]:
nginx_monitor = ServiceMonitor("nginx", 80)
print(isinstance(nginx_monitor, ServiceMonitor))   # check if the object is instance of the specified class

redis_monitor = ServiceMonitor(service_name="redis", port=6379)
print(isinstance(redis_monitor, ServiceMonitor))

print(nginx_monitor.service)
print(redis_monitor.service)

Initializing monitor for service nginx on port 80.
True
Initializing monitor for service redis on port 6379.
True
nginx
redis


## Instance Methods: Object Behavior
- **Definition:** Functions defined *inside* a class definition.
- **First Parameter:** Always `self` (by strong convention), allowing the method to access and modify the instance's attributes (`self.attribute_name`).
- **Calling:** Use dot notation on an instance: `instance.method_name(arguments)`. Python automatically passes the instance (`instance`) as the `self` argument.

In [11]:
class ServiceMonitor:
    """Provides service checks for a single service"""
    def __init__(self, service_name, port):
        """Initializes the monitor for a specific service.

        Args:
            service_name (str): the name of the service.
            port (int): the port to use for checks.
        """
        print(f"Initializing monitor for service {service_name} on port {port}.")
        self.service = service_name
        self.port = port
        self.is_alive = False

    def check(self):
        """Simulates checking the service status"""
        print(f"METHOD: Checking {self.service} on port {self.port}...")
        self.is_alive = True
        print(f"METHOD: Status for service {self.service}: {"Alive" if self.is_alive else "Down"}")
        return self.is_alive

nginx_monitor = ServiceMonitor("nginx", 80)
status = nginx_monitor.check()
print(f"Received status: {status}")

Initializing monitor for service nginx on port 80.
METHOD: Checking nginx on port 80...
METHOD: Status for service nginx: Alive
Received status: True


## Basic Inheritance: Reusing and Extending
- **Concept:** Create a new class (Child/Subclass) that inherits properties (attributes and methods) from an existing class (Parent/Superclass). Promotes code reuse (DRY).
- **Syntax:** `class ChildClassName(ParentClassName):`
- **Inherited Members:** The Child automatically gets all methods and attributes defined in the Parent.
- **Specializing:** The Child can:
  - Add *new* attributes and methods.
  - *Override* parent methods by defining a method with the same name.
- **`super()`:** Inside the Child's methods, use `super().method_name(...)` to explicitly call the Parent's version of a method (very common in `__init__`).

In [12]:
class HttpServiceMonitor(ServiceMonitor):
    """Extends ServiceMonitor to add an HTTP endpoint check."""
    def __init__(self, service_name, port, url):
        super().__init__(service_name, port)
        self.url = url

    def ping(self):
        """Ping url provided when creating instance."""
        print(f"METHOD: Pinging url {self.url}")

    def check(self):
        alive = super().check()
        print(f"METHOD: Performing HTTP check on {self.url}")

http_monitor = HttpServiceMonitor("web", 8080, "http://localhost")
nginx_monitor = ServiceMonitor("nginx", 80)
http_monitor.ping()
http_monitor.check()
# nginx_monitor.ping() # Uncommenting will raise AttributeError since ping() is a method only of the subclass
nginx_monitor.check()

Initializing monitor for service web on port 8080.
Initializing monitor for service nginx on port 80.
METHOD: Pinging url http://localhost
METHOD: Checking web on port 8080...
METHOD: Status for service web: Alive
METHOD: Performing HTTP check on http://localhost
METHOD: Checking nginx on port 80...
METHOD: Status for service nginx: Alive


True

## Data Classes
Data class is a simple way to create classes that store data.
It comes with Python standard library.

In [8]:
from dataclasses import dataclass

@dataclass
class Item:
    """
    Data class automatically implements the following methods behind the scene:
        - __init__
        - __repr__
        - __eq__
    """

    name: str
    unit_price: float
    quantity: int = 0

    def total_cost(self) -> float:
        return self.unit_price * self.quantity



item1 = Item("Monitor", 45.50, 1)

# Auto implements __repr__
print(item1)
print(repr(item1))

# Auto implements __eq__
item2 = Item("Monitor", 45.50, 1)
print(f"Are equal: {item1 == item2}")

Item(name='Monitor', unit_price=45.5, quantity=1)
Item(name='Monitor', unit_price=45.5, quantity=1)
Are equal: True


## Abstract Base Class

In [9]:
from abc import ABC, abstractmethod


# Abstract base class for payment providers
class PaymentProvider(ABC):
    @abstractmethod
    def process_payment(self, customer_id: str, amount: float) -> str:
        pass


# Paypal implementation
class Paypal(PaymentProvider):
    def process_payment(self, customer_id: str, amount: float) -> str:
        return (
            f"Processed payment of ${amount:.2f} for customer {customer_id} via PayPal"
        )


# Stripe implementation
class Stripe(PaymentProvider):
    def process_payment(self, customer_id: str, amount: float) -> str:
        return (
            f"Processed payment of ${amount:.2f} for customer {customer_id} via Stripe"
        )


# Customer class
class Customer:
    def __init__(self, customer_id: str, name: str) -> None:
        self.customer_id = customer_id
        self.name = name

    # `make_payment` does not need to know anything about the PaymentProvider implementation
    def make_payment(self, provider: PaymentProvider, amount: float):
        print(provider.process_payment(self.customer_id, amount))


# Main function
def main() -> None:
    # Create customers
    customer1 = Customer("C123", "Jim")
    customer2 = Customer("C456", "Kale")

    # Create payment providers
    paypal = Paypal()
    stripe = Stripe()

    # Customers make payments
    customer1.make_payment(paypal, 100.0)
    customer2.make_payment(stripe, 200.0)


main()

Processed payment of $100.00 for customer C123 via PayPal
Processed payment of $200.00 for customer C456 via Stripe


# Class attributes vs Instance attributes

- **Class attributes:** belong to the class itself and will be shared by all instances. Such attributes are defined in the top of the body.
- **Instance attributes:** belong to ONLY a particular instance of the class. It is not shared by all instances. Every object has its own copy of the instance attribute.  

In [ ]:
class SampleClass:
    count = 0   # Class attribute

    def increment(self):
        SampleClass.count += 1 


s1 = SampleClass()
s1.increment()

s2 = SampleClass()
s2.increment()
print(f"Current count: {SampleClass.count}") # access class attribute using Class itself



class DemoClass:
    def __init__(self, id: int, name: str) -> None:
        # Instance attributes
        self.id = id
        self.name = name


d1 = DemoClass(10, 'Jim')
print(f"Dictionary from: {vars(d1)}")


Current count: 2
Dictionary from: {'id': 10, 'name': 'Jim'}


## Instance methods vs Class methods vs Static methods
- **Instance methods:** use a `self` parameter pointing to an instance of the class. They can access and modify instance state through `self.__class__`
- **Class methods:** use a `cls` parameter pointing to the class itself. They can modify class level state but can't modify individual instance state
- **Static methods:** don't take `self` or `cls` parameters. They can't modify instance or class state directly. 

In [ ]:
class DemoClass:
    def instance_method(self):
        return f"Instance method called {self}"
    
    @classmethod
    def class_method(cls):
        return f"Class method called {cls}" 

    @staticmethod
    def static_method():
        return f"Static method called" 

## Hands-on Exercise

In [1]:
from datetime import datetime, timezone

class Account:
    """Simple account class with balance"""

    # Fields and methods starting with _ are meant for internal use only
    # similar to private fields and methods even though it is NOT Strictly
    # enforced by Python.
    @staticmethod
    def _current_time():
        utc_time = datetime.now(tz=timezone.utc)
        return utc_time.astimezone()  # localize time

    def __init__(self, name, balance):
        self._name = name  # internal field

        # Python performs name mangling for fields starting with dunder
        # by prefixing Class name to the method name.
        # In this case, __balance will be converted to _Account__balance
        # So, when we use the field inside the class it refers to correct field name.
        # But when this field is referred outside the class, the correct fiels name is hidden.
        # This is done to prevent accident change of the fields from outside the class.
        self.__balance = balance

        self._transaction_list = [(Account._current_time(), balance)]  # internal field
        print(f"Account created for {self._name}")

    # ------------------ Public Methods
    def deposit(self, amount):
        if amount > 0:
            self.__balance += amount
            self._transaction_list.append((Account._current_time(), amount))

    def withdraw(self, amount):
        if 0 < amount <= self.__balance:
            self.__balance -= amount
            self._transaction_list.append((Account._current_time(), -amount))
        else:
            print("The amount must be greater than zero or less than your balance")

    def show_balance(self):
        print(f"Balance is {self.__balance}")

    def show_transactions(self):
        for date, amount in self._transaction_list:
            if amount > 0:
                transaction_type = "deposited"
            else:
                transaction_type = "withdrawn"
            print(f"{amount:6} was {transaction_type} on {date}")


jim = Account("Jim", 2000)
jim.show_balance()
jim.deposit(1000)
jim.show_balance()
jim.withdraw(500)
jim.show_balance()
jim.show_transactions()

jim.__balance = 0  # this won't refer to the instance property __balance
jim.show_balance()


Account created for Jim
Balance is 2000
Balance is 3000
Balance is 2500
  2000 was deposited on 2025-07-23 16:27:02.537317+01:00
  1000 was deposited on 2025-07-23 16:27:02.537478+01:00
  -500 was withdrawn on 2025-07-23 16:27:02.537509+01:00
Balance is 2500


In [ ]:
class VM:
    # Class attribute (value shared by all instances)
    hypervisor = "KVM"

    def __init__(self, name) -> None:
        self.name = name

vm1 = VM('web-01')
vm2 = VM('db-01')

VM.hypervisor = "Xen"

print(f"{vm1.name} on {vm1.hypervisor}, {vm2.name} on {vm2.hypervisor} ")

web-01 on Xen, db-01 on Xen 
